In [2]:
# Dependencies
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets 
import pandas as pd
import time
import requests
from census import Census
#pd.set_option('display.max_rows', 30000)
# Census API Key
from config import census_key
c = Census(census_key, year=2015)

## Retrieve Crime Data from FBI Crime Database - UCR

In [4]:
#Store file path for crime data in a variable
crime_data = "../Crime Source Data/Offenses_Known_to_Law_Enforcement_by_State_by_City_2015.xls"

#Read data into dataframe
crime_data_df=pd.read_excel(crime_data)

#Extract the state into a list so all rows in "state" column can have a state value
state = crime_data_df["State"]
states=[]
for s in state:
    if pd.isnull(s):
        states.append(current_state)
    else:
        current_state = s
        states.append(current_state)
        
#Add newly created list into the dataframe and remove the old column
crime_data_df["NEW_STATE"] = states
clean_crime_data_df=crime_data_df[["NEW_STATE", "City", "Population", "Violent Crime"]] 
clean_crime_data_df=clean_crime_data_df.rename(columns={"NEW_STATE":"State"})

#Remove any trailing numbers at the end of state and city names
clean_crime_data_df["State"] = clean_crime_data_df["State"].map(lambda x: x.rstrip("1234567890"))
clean_crime_data_df["City"] = clean_crime_data_df["City"].map(lambda x: x.rstrip("1234567890"))

#Calculate the rate of violent crime per capita and remove any city where the violent crime is 0
clean_crime_data_df = clean_crime_data_df.loc[clean_crime_data_df["Violent Crime"]>0]
clean_crime_data_df["Violent Crime Rate"] = (clean_crime_data_df["Violent Crime"]/clean_crime_data_df["Population"])*100

clean_crime_data_df=clean_crime_data_df.sort_values("State")
clean_crime_data_df = clean_crime_data_df.loc[clean_crime_data_df["Violent Crime"]>0]

## Sort Out the Highest and Lowest Crime Rate for Each State

In [6]:
#Sort out the highest crime rate for each state
#variable to hold highest violent crime for each state
violent_crime=[]

#variable to track highest violent crime for each state
violent_crime_track=0

#variable to hold starting state for the loop
current_state='ALABAMA'

#variable to append city for highest crime rate
city=[]

#variable to append city for highest crime rate
state=[]

#Loop through each row in dataframe 
for index,row in clean_crime_data_df.iterrows():
    if row['State'] == current_state:
        if row["Violent Crime Rate"]> violent_crime_track:
            highest = row["Violent Crime Rate"] 
            cur_city = row["City"]
            cur_state = row["State"]
            violent_crime_track = highest
               
    else:
        violent_crime.append(highest)
        highest = row["Violent Crime Rate"]
        violent_crime_track = highest
        current_state=row['State']
        city.append(cur_city)
        state.append(cur_state)
        cur_city = row["City"]
        cur_state = row["State"]
#append the data for the last state
violent_crime.append(highest)
city.append(cur_city)
state.append(cur_state)

#Place newly created lists into dictionary and convert to DataFrame
highest_new_dict = {'State':state,'City':city,'Violent Crime Rate':violent_crime}
highest_new_df = pd.DataFrame(highest_new_dict)
highest_new_df.to_csv("../Analysis/highest_crime.csv", index=False)

,State,City,Violent Crime Rate
0,ALABAMA,Anniston,2.721241
1,ALASKA,Kotzebue,2.633172
2,ARIZONA,Willcox,2.325581
3,ARKANSAS,Osceola,2.063140
4,CALIFORNIA,Industry,33.009709
5,COLORADO,Lakeside,12.500000
6,CONNECTICUT,Hartford,1.140880
7,DELAWARE,Dewey Beach,4.054054
8,DISTRICT OF COLUMBIA,Washington,1.202568
9,FLORIDA,Florida City,3.155577


In [5]:
#Sort out the lowest crime rate for each state
#variable to hold lowest violent crime for each state
violent_crime=[]

#variable to track lowest violent crime for each state
violent_crime_track=100

#variable to hold starting state for the loop
current_state='ALABAMA'

#variable to append city for lowest crime rate
city=[]

#variable to append city for lowest crime rate
state=[]

#Loop through each row in dataframe 
for index,row in clean_crime_data_df.iterrows():
    if row['State'] == current_state:
        if row["Violent Crime Rate"]< violent_crime_track:
            lowest = row["Violent Crime Rate"] 
            cur_city = row["City"]
            cur_state = row["State"]
            violent_crime_track = lowest
               
    else:
        violent_crime.append(lowest)
        lowest = row["Violent Crime Rate"]
        violent_crime_track = lowest
        current_state=row['State']
        city.append(cur_city)
        state.append(cur_state)
        cur_city = row["City"]
        cur_state = row["State"]
        
#append the data for the last state
violent_crime.append(lowest)
city.append(cur_city)
state.append(cur_state)

#Place newly created lists into dictionary and convert to DataFrame
lowest_new_dict = {'State':state,'City':city,'Violent Crime Rate':violent_crime}
lowest_new_df = pd.DataFrame(lowest_new_dict)
lowest_new_df.to_csv("../Analysis/lowest_crime.csv", index=False)

,State,City,Violent Crime Rate
0,ALABAMA,Argo,0.023730
1,ALASKA,Wrangell,0.042535
2,ARIZONA,Buckeye,0.045362
3,ARKANSAS,Greenbrier,0.018730
4,CALIFORNIA,Hillsborough,0.008651
5,COLORADO,Frederick,0.008648
6,CONNECTICUT,Wilton,0.005305
7,DELAWARE,Ocean View,0.049456
8,DISTRICT OF COLUMBIA,Washington,1.202568
9,FLORIDA,Parkland,0.023938
